In [1]:
import torch
from torch import nn
from torch import Tensor
import requests
import os
from pathlib import Path
from typing import Callable
from functools import partial


URL = 'https://download.pytorch.org/models/resnet18-5c106cde.pth'

In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
from glasses.nn.models.classification.efficientnet import SEInvertedResidualBlock, EfficientNetEncoder


SEInvertedResidualBlock(32, 64)

SEInvertedResidualBlock(
  (block): Residual(
    (block): Sequential(
      (exp): ConvBnAct(
        (conv): Conv2dPad(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): Swish()
      )
      (conv): Sequential(
        (0): ConvBnAct(
          (conv): DepthWiseConv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=192, bias=False)
          (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
        (1): Sequential(
          (0): SEModuleConv(
            (avg_pool): AdaptiveAvgPool2d(output_size=1)
            (att): Sequential(
              (0): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): Swish()
              (2): Conv2d(48, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (3): Sigmoid()
            )
          )
        

In [5]:
EfficientNetEncoder()

EfficientNetEncoder(
  (gate): Sequential(
    (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blocks): ModuleList(
    (0): EfficientNetLayer(
      (block): Sequential(
        (0): SEInvertedResidualBlock(
          (block): Residual(
            (block): Sequential(
              (conv): Sequential(
                (0): ConvBnAct(
                  (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                  (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (act): Swish()
                )
                (1): Sequential(
                  (0): SEModuleConv(
                    (avg_pool): AdaptiveAvgPool2d(output_size=1)
                    (att): Sequential(
                      (0): Conv2d(32, 1, kernel_size=(1, 1), stride=(1, 1

In [6]:
from efficientnet_pytorch import EfficientNet
from torchsummary import summary


model = EfficientNet.from_pretrained('efficientnet-b0')

summary(model.cuda(), (3,224,224))

Loaded pretrained weights for efficientnet-b0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 226, 226]               0
Conv2dStaticSamePadding-2         [-1, 32, 112, 112]             864
       BatchNorm2d-3         [-1, 32, 112, 112]              64
MemoryEfficientSwish-4         [-1, 32, 112, 112]               0
         ZeroPad2d-5         [-1, 32, 114, 114]               0
Conv2dStaticSamePadding-6         [-1, 32, 112, 112]             288
       BatchNorm2d-7         [-1, 32, 112, 112]              64
MemoryEfficientSwish-8         [-1, 32, 112, 112]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

In [14]:
model

EfficientNet(
  (encoder): EfficientNetEncoder(
    (gate): Sequential(
      (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (blocks): ModuleList(
      (0): EfficientNetLayer(
        (block): Sequential(
          (0): SEInvertedResidualBlock(
            (block): Residual(
              (block): Sequential(
                (conv): Sequential(
                  (0): ConvBnAct(
                    (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (act): Swish()
                  )
                  (1): Sequential(
                    (0): SEModuleConv(
                      (avg_pool): AdaptiveAvgPool2d(output_size=1)
                      (att): Sequential(
      

In [13]:
from glasses.nn.models.classification.efficientnet import EfficientNet

model  = EfficientNet()


summary(model.cuda(), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
   DepthWiseConv2d-3         [-1, 32, 112, 112]             288
       BatchNorm2d-4         [-1, 32, 112, 112]              64
             Swish-5         [-1, 32, 112, 112]               0
 AdaptiveAvgPool2d-6             [-1, 32, 1, 1]               0
            Conv2d-7              [-1, 4, 1, 1]             128
             Swish-8              [-1, 4, 1, 1]               0
            Conv2d-9             [-1, 32, 1, 1]             128
          Sigmoid-10             [-1, 32, 1, 1]               0
     SEModuleConv-11         [-1, 32, 112, 112]               0
        Conv2dPad-12         [-1, 16, 112, 112]             512
      BatchNorm2d-13         [-1, 16, 112, 112]              32
         Residual-14         [-1, 16, 1

In [15]:
model

EfficientNet(
  (encoder): EfficientNetEncoder(
    (gate): Sequential(
      (conv): Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (blocks): ModuleList(
      (0): EfficientNetLayer(
        (block): Sequential(
          (0): SEInvertedResidualBlock(
            (block): Residual(
              (block): Sequential(
                (conv): Sequential(
                  (0): ConvBnAct(
                    (conv): DepthWiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (act): Swish()
                  )
                  (1): Sequential(
                    (0): SEModuleConv(
                      (avg_pool): AdaptiveAvgPool2d(output_size=1)
                      (att): Sequential(
      